imports

In [55]:
import pandas as pd
import numpy as np
import pathlib
import os
from PIL import Image
import matplotlib.pyplot as plt
from characterization_ams.stats_engine import stats
from characterization_ams.emva import emva
from characterization_ams.standard_tests import ptc
from IPython.display import display


image location

In [19]:
dark_images = pathlib.Path(r'C:\Users\pbae.OFFICE\Documents\MEGAsync\ams\mira050dark_25')
results=dark_images/'results.csv'

In [119]:
df=pd.read_csv(results)
with pd.option_context('display.max_rows', 5,
                       'display.max_columns', 8,
                       'display.precision', 3,
                       'display.max_colwidth', 30
                       ):
    display(df)

,bitmode,again,dgain,exp_us,filename,testcase,temperature,amount
0,10bit,1,1,125,10bit_again_1_dgain_1_exp_...,dark_25,25,15
1,10bit,1,1,200,10bit_again_1_dgain_1_exp_...,dark_25,25,15
...,...,...,...,...,...,...,...,...
8,10bit,16,1,125,10bit_again_16_dgain_1_exp...,dark_25,25,15
9,10bit,16,1,200,10bit_again_16_dgain_1_exp...,dark_25,25,15


In [75]:
ptclist=[]
imnames=df['filename']
bitmode=df['bitmode'][0]
bpp=int(bitmode.split('bit')[0])
print(bpp)


10


In [116]:
df_temp = pd.DataFrame()

for imname in imnames:
    images=[]
    for imname2 in dark_images.glob(f'./{imname}*.tiff'):
        im = Image.open(imname2)
        images.append(np.array(im))
    images=np.array(images)
    images=images/(2**(16-bpp))
    ptclist = [i for i in ptclist]
    ptclist.append(images)
    agres = stats.agg_results(images)  
    df_temp = df_temp.append(agres, ignore_index = True)


df.rename(columns = {'mean':'meanold'}, inplace = True)
statistics=ptc.get_stats(ptclist, df=df)
# ptc.ptc(ptclist, df, 'exp_us')

In [125]:
res=df.join(df_temp)
display(res)

,bitmode,again,dgain,exp_us,filename,testcase,temperature,amount,Mean [DN],Total FPN [DN],Pix FPN [DN],Col FPN [DN],Row FPN [DN],Total Temp Noise [DN],Pix Temp Noise [DN],Col Temp Noise [DN],Row Temp Noise [DN]
0,10bit,1,1,125,10bit_again_1_dgain_1_exp_us_ 125_dark_25__tem...,dark_25,25,15,37.279,0.329,0.297,0.089,0.111,0.829,0.724,0.082,0.396
1,10bit,1,1,200,10bit_again_1_dgain_1_exp_us_ 200_dark_25__tem...,dark_25,25,15,37.264,0.331,0.297,0.090,0.117,0.830,0.720,0.108,0.398
2,10bit,2,1,125,10bit_again_2_dgain_1_exp_us_ 125_dark_25__tem...,dark_25,25,15,43.535,0.666,0.594,0.186,0.238,1.448,1.205,0.175,0.785
3,10bit,2,1,200,10bit_again_2_dgain_1_exp_us_ 200_dark_25__tem...,dark_25,25,15,43.478,0.669,0.594,0.185,0.246,1.446,1.202,0.195,0.782
4,10bit,4,1,125,10bit_again_4_dgain_1_exp_us_ 125_dark_25__tem...,dark_25,25,15,45.584,1.376,1.209,0.395,0.526,2.817,2.224,0.428,1.678
5,10bit,4,1,200,10bit_again_4_dgain_1_exp_us_ 200_dark_25__tem...,dark_25,25,15,45.547,1.378,1.214,0.394,0.521,2.809,2.219,0.459,1.662
6,10bit,8,1,125,10bit_again_8_dgain_1_exp_us_ 125_dark_25__tem...,dark_25,25,15,63.025,2.638,2.343,0.761,0.949,5.598,4.486,0.899,3.231
7,10bit,8,1,200,10bit_again_8_dgain_1_exp_us_ 200_dark_25__tem...,dark_25,25,15,62.772,2.633,2.352,0.762,0.909,5.557,4.524,0.672,3.162
8,10bit,16,1,125,10bit_again_16_dgain_1_exp_us_ 125_dark_25__te...,dark_25,25,15,82.211,5.289,4.420,2.207,1.898,10.914,8.546,1.676,6.587
9,10bit,16,1,200,10bit_again_16_dgain_1_exp_us_ 200_dark_25__te...,dark_25,25,15,81.973,5.288,4.425,2.206,1.884,10.868,8.568,1.523,6.519


In [131]:
res[res["again"] == 1]

,bitmode,again,dgain,exp_us,filename,testcase,temperature,amount,Mean [DN],Total FPN [DN],Pix FPN [DN],Col FPN [DN],Row FPN [DN],Total Temp Noise [DN],Pix Temp Noise [DN],Col Temp Noise [DN],Row Temp Noise [DN]
0,10bit,1,1,125,10bit_again_1_dgain_1_exp_us_ 125_dark_25__tem...,dark_25,25,15,37.279,0.329,0.297,0.089,0.111,0.829,0.724,0.082,0.396
1,10bit,1,1,200,10bit_again_1_dgain_1_exp_us_ 200_dark_25__tem...,dark_25,25,15,37.264,0.331,0.297,0.090,0.117,0.830,0.720,0.108,0.398
